In [1]:
# !pip install dask[complete] dask-ml
# !pip install gensim[summarization]
# !pip install gensim==3.8.3
# !pip install sumy

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


In [3]:
import os

os.chdir('..') # Change to parent directory
cwd = os.getcwd() # Get current working directory
print(cwd)

C:\Users\templ\OneDrive\Desktop\AI\Hackathon\Amazon


In [4]:
# Read the dataset using Dask
dataset = pd.read_csv('dataset/universal_train.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
# dataset =  dataset.sample(n=20000)

In [5]:
dataset.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH', 'PRODUCT_LENGTH_NONORM'],
      dtype='object')

In [6]:
dataset.drop(['Unnamed: 0'],axis=1)

,0,1,2,3,4,5,6,7,8,9,10,PRODUCT_TYPE_ID,PRODUCT_LENGTH,PRODUCT_LENGTH_NONORM
0,0.033740,0.053593,0.119451,-0.002144,-0.023888,-0.009354,-0.013213,-0.011067,-0.052480,-0.005460,-0.048928,-0.592630,-0.001440,2125.980000
1,0.021798,0.017311,-0.010543,0.044631,-0.013809,0.007838,-0.019112,0.012684,-0.002693,0.000218,0.007681,-0.314022,-0.002721,393.700000
2,0.021713,0.010882,0.012488,-0.015654,0.011082,0.004986,-0.017478,-0.003682,-0.005930,-0.000192,-0.005423,0.891683,-0.002459,748.031495
3,0.041322,0.102819,-0.029833,0.067824,0.101342,-0.023887,0.054705,-0.005680,-0.031651,-0.000720,-0.008348,-0.253258,-0.002430,787.401574
4,0.001786,0.004194,0.001742,-0.000216,-0.001976,0.006235,-0.000607,0.000235,-0.002081,0.000451,0.008637,0.532392,-0.002570,598.424000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249693,0.019544,0.061277,-0.024850,0.036257,0.073269,-0.017734,0.047925,0.000918,0.012451,-0.000118,-0.004641,-0.249980,-0.002139,1181.100000
2249694,0.020147,0.015623,0.021427,-0.000404,0.003816,0.009220,-0.025823,-0.002782,-0.011322,0.000861,0.017507,-0.148118,-0.002919,125.984252
2249695,0.063208,0.032880,0.119688,0.024788,-0.071651,-0.024792,0.045420,-0.009083,0.004295,-0.002458,-0.022181,-0.611792,-0.002125,1200.000000
2249696,0.033887,0.054742,0.086165,-0.033040,-0.014690,-0.016088,0.023324,0.032273,0.099026,-0.000348,-0.022752,-0.859387,-0.000867,2900.000000


In [7]:
df = dataset

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from scipy.sparse import hstack


In [18]:
# Split the data into features (X) and target (y)
X = df.drop(columns = ['Unnamed: 0','PRODUCT_TYPE_ID','PRODUCT_LENGTH','PRODUCT_LENGTH_NONORM'], axis=1)
y = df['PRODUCT_LENGTH']

In [19]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [21]:
# Convert y_train to a 1D array
y_train = y_train.values.ravel()

In [22]:
# Select the most relevant features using SelectKBest
selector = SelectKBest(f_regression, k=5)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_val_selected = selector.transform(X_val_scaled)

In [24]:
X_train_selected

array([[-4.57684152e-01,  3.18890015e-02,  4.39176919e-03,
        -1.65307492e-01, -4.19713330e-01],
       [ 5.55979130e-01, -1.04476268e-01, -4.68948539e+00,
        -2.69864890e+00,  1.88605766e-01],
       [ 5.82221092e-02, -3.88482447e-01, -1.15624057e-01,
        -1.50577259e-01, -4.27852246e-01],
       ...,
       [ 1.61940825e-01,  1.20136722e+00,  8.64063904e-01,
        -1.69113464e+00,  3.63634005e-01],
       [ 3.93410225e-01, -1.26274496e+00,  5.44963215e-01,
        -9.59446095e-02, -3.46188115e-01],
       [ 1.17519790e+00, -1.33161005e+00,  1.12092143e+00,
         1.76205884e-01, -1.03333623e+00]])

In [25]:
# Train a linear regression model on the selected features
model = LinearRegression()
model.fit(X_train_selected, y_train)

LinearRegression()

In [26]:
# Predict on the validation set and calculate RMSE
y_pred = model.predict(X_val_selected)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f'RMSE: {rmse:.2f}')

RMSE: 0.47


In [27]:
import pickle
# Save the model to a file
with open('AmazonLRmodel.pickle', 'wb') as f:
    pickle.dump(model, f)

In [28]:
testdataset = pd.read_csv('dataset/universal_test.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
# dataset =  dataset.sample(n=20000)

In [29]:
testdataset.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       'PRODUCT_TYPE_ID'],
      dtype='object')

In [30]:
testdataset = testdataset.drop(['Unnamed: 0',"PRODUCT_TYPE_ID"],axis=1)
testdataset.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.002030,0.001105,0.004455,-0.002957,0.081613,-0.026087,-0.060922,-0.002510,-0.000102,0.002701,0.000682
1,0.045924,0.004526,0.038838,0.000920,-0.004138,-0.004475,0.000287,-0.023393,-0.012571,-0.010897,-0.031862
2,0.036662,0.006406,0.021295,-0.007004,0.011642,0.054653,-0.005702,0.005560,-0.000605,-0.006511,0.050248
3,0.014183,0.006882,0.013506,0.006704,0.003102,0.003619,0.002005,-0.013562,-0.004010,0.003927,-0.002901
4,0.001468,0.000477,0.001944,-0.000566,0.002886,-0.000283,0.001136,-0.001220,0.000452,0.002904,0.000571


In [41]:
# Select the relevant columns from the test dataset
testdataset = testdataset.iloc[:, [0, 1, 2, 3, 4]]


# Make predictions using the selected features
y_pred = model.predict(testdataset)

# Put the predictions in the PRODUCT_LENGTH column
testdataset['PRODUCT_LENGTH'] = y_pred

# # Select the relevant columns for output
# output = testdataset[['PRODUCT_TYPE_ID', 'PRODUCT_LENGTH']]

# Save the output as a CSV file
testdataset.to_csv('output.csv', index=False)

C:\Users\templ\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [47]:
y_pred

array([6.98631843e-06, 5.76673436e-05, 4.68544215e-05, ...,
       6.43533424e-05, 1.29886011e-04, 4.55691869e-05])

In [48]:
y_pred1 = y_pred*dataset["PRODUCT_LENGTH_NONORM"].std()  + dataset["PRODUCT_LENGTH_NONORM"].mean()

In [49]:
y_pred1

array([4081.28251722, 4149.78728606, 4135.1716242 , ..., 4158.82464867,
       4247.40415763, 4133.4343922 ])

In [39]:
#test = pd.read_csv('dataset/test.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
test = test.drop(columns = ['PRODUCT_TYPE_ID','TITLE','BULLET_POINTS','DESCRIPTION'],axis = 1)
test

,PRODUCT_ID
0,604373
1,1729783
2,1871949
3,1107571
4,624253
...,...
734731,921419
734732,2456362
734733,841529
734734,1190194


In [50]:
test["PRODUCT_LENGTH"] = y_pred1
test

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,4081.282517
1,1729783,4149.787286
2,1871949,4135.171624
3,1107571,4149.773828
4,624253,4153.515508
...,...,...
734731,921419,4132.525075
734732,2456362,4149.535864
734733,841529,4158.824649
734734,1190194,4247.404158


In [59]:
test["PRODUCT_LENGTH"] = test["PRODUCT_LENGTH"] - 3600
test

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,481.282517
1,1729783,549.787286
2,1871949,535.171624
3,1107571,549.773828
4,624253,553.515508
...,...,...
734731,921419,532.525075
734732,2456362,549.535864
734733,841529,558.824649
734734,1190194,647.404158


In [60]:
test.to_csv("Namuna3600.csv", index = False)